In [1]:
import os
import re
import fnmatch
import pandas as pd
import numpy as np
import warnings
import split_extract
from multiprocessing import Pool
from tqdm import tqdm
from params import dresden_images_root, images_db_path, patch_span, \
        patch_num, robust_root, robust_db_path

In [2]:
warnings.filterwarnings("ignore")

if not os.path.exists(robust_root):
    os.makedirs(robust_root)

In [3]:
images_db = np.load(images_db_path, allow_pickle=True).item()

test_model_list = list(filter(lambda model: re.search(r'^((?!Canon).)*$', model), images_db['brand_model']))
test_img_list = list(filter(lambda img: re.search(r'^((?!Canon).)*$', img), images_db['path']))

# How many model in my dataset
model_list = np.unique(test_model_list) 

In [4]:
for model in model_list:
    tmp_list = fnmatch.filter(test_model_list, model + '*')
    print("{} in test set: {}.".format(model, len(tmp_list)))

Nikon_CoolPixS710 in test set: 925.
Nikon_D200 in test set: 752.
Nikon_D70 in test set: 736.
Nikon_D70s in test set: 367.
Sony_DSC-H50 in test set: 541.
Sony_DSC-T77 in test set: 725.
Sony_DSC-W170 in test set: 405.


In [5]:
test_list = []
for img_brand_model, img_path in \
    tqdm(zip(test_model_list, test_img_list)):
        test_list += [{'data_set':'test',
                       'img_path':img_path,
                       'img_brand_model':img_brand_model,
                       'patch_span':patch_span,
                       'patch_num':patch_num,
                       'patch_root': robust_root,
                       'img_root': dresden_images_root
                       }]

num_processes = 12
# num_processes = 4
pool = Pool(processes=num_processes)

4084it [00:00, 1359810.87it/s]


In [6]:
test_paths = pool.map(split_extract.extract, test_list)
print('Completed.')

Completed.
